In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
data=pd.read_csv(r'C:\Users\Facundo Toneguzzo\Scrapper_Portal\data\raw\2024\202407.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9217 entries, 0 to 9216
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                9217 non-null   object 
 1   Name of the flat   9217 non-null   object 
 2   Value              9217 non-null   int64  
 3   Currency           9217 non-null   object 
 4   General Expenses   8236 non-null   float64
 5   Size of the flat   9146 non-null   float64
 6   Bedrooms           9191 non-null   float64
 7   Bathrooms          9198 non-null   float64
 8   Seller             9217 non-null   object 
 9   metraje            9146 non-null   float64
 10  sup_terraza        5028 non-null   float64
 11  sup_util           9081 non-null   float64
 12  ambientes          4248 non-null   float64
 13  dormitorios        8792 non-null   float64
 14  banos              6953 non-null   float64
 15  estacionamiento    5961 non-null   float64
 16  bodegas            4196 

In [4]:
map_attrib=[column for column in data.columns if data[column].dtype=='object']

In [5]:
data['orientacion'].value_counts().to_dict()

{'Nor-Oriente': 329,
 '-': 303,
 'SO': 302,
 'Nor-Poniente': 291,
 'N': 274,
 'O': 273,
 'SP': 251,
 'P': 239,
 'NP': 223,
 'Sur-Oriente': 210,
 'Oriente': 177,
 'Norte': 150,
 'S': 132,
 'NOSP': 127,
 'Sur': 116,
 'Sur-Poniente': 82,
 'Todas': 65,
 'Nor-Orienteff': 57,
 'Poniente': 46,
 'NOP': 45,
 'NorPoniente-Sur': 35,
 'NOS': 24,
 'Todas Las Anteriores': 21,
 'Norte-sur': 20,
 'Sur-Oriente-Poniente': 18,
 'OSP': 8,
 'NORORIENTE': 7,
 'NorOriente-Sur': 6,
 'Sur-oriente': 6,
 'PON-SUR': 6,
 'Nor-poniente': 5,
 'Oriente Sur': 5,
 'NorOriente-Poniente': 4,
 'Nor / Oriente / Poniente': 4,
 'NPS': 3,
 'Nor-Oriente-Poniente': 3,
 'Oriente-SurPoniente': 3,
 'Norponiente': 2,
 'SUR': 2,
 'Nor-oriente': 2,
 'ORIENTE': 1,
 '1': 1,
 'Nor-Oriente-Sur': 1,
 'SURORIENTE': 1}

In [6]:
orientacion={'N':['norte','n'],
'NO':['nor-oriente','nor-orienteff','nororiente','no','on'],
'NP':['norponiente','nor-poniente','np','pn'],
'S':['sur','s'],
'SO':['sur-oriente','oriente sur','so','os'],
'SP':['sur-poniente','pon-sur','sp','ps'],
'O':['oriente','o'],
'P':['poniente','p'],
'NS':['norte-sur','ns','sn'],
'OP':['oriente-poniente','op','po'],
'NOP':['nororiente-poniente','nor-oriente-poniente','nor / oriente / poniente','nop','npo','pno','pon','opn','onp'],
'SOP':['sur-oriente-poniente','osp','oriente-surponiente'],
'NSO':['nororiente-sur','nos','nor-oriente-sur'],
'NSP':['norponiente-sur','nps'],
'NOSP':['todas','todas las anteriores','nosp'],
np.nan:['1','-'],    
}

In [7]:
def map_orient(value):
    if pd.isna(value):
        return np.nan
    value_lower = str(value).lower()
    for categoria, keywords in orientacion.items():
        if value_lower in keywords: 
            return categoria
    return value

In [8]:
data['new_orient']=data['orientacion'].map(map_orient)

In [9]:
data['new_orient'].unique()

array([nan, 'S', 'SO', 'N', 'SP', 'P', 'NP', 'O', 'NO', 'NOSP', 'NS',
       'NSO', 'NOP', 'SOP', 'NSP', 'SURORIENTE'], dtype=object)

In [10]:
depa={'Departamento':['departamento','habitacional','completo'],
    'Piso':['piso'],
    'Semi Piso':['semi piso','semi-piso'],
    'Clasico':['clásico','clasico','tradicional','cocina tradicional'],
    'Moderno':['moderno','cocina incorporada'],
    'Monoambiente':['monoambiente','un ambiente','solo un ambiente'],
    'Penthouse':['penthhouse','penhouse','penthouse','penthouse duplex'],
    'Standar':['standar','estándar'],
    'Normal':['normal'],
    'Duplex':['duplex','dúplex','dúpex'],
    'Loft':['loft'],
    'Triplex':['triplex','tríplex'],
    'Oficina':['apto oficina','oficina'],
    'Estudio':['studio','estudio'],
    np.nan:['-']
}

In [11]:
def map_depa(value):
    if pd.isna(value):
        return np.nan
    value_lower = str(value).lower()
    for categoria, keywords in depa.items():
        if value_lower in keywords: 
            return categoria
    return str('review_'+value)

In [12]:
data['map_depa']=data['tipo_depa'].map(map_depa)

In [13]:
data['map_depa'].unique()

array(['Departamento', nan, 'Piso', 'Semi Piso', 'Clasico', 'Penthouse',
       'Duplex', 'Monoambiente', 'Moderno', 'Normal', 'Triplex',
       'Standar', 'Loft', 'Oficina', 'Estudio'], dtype=object)

In [14]:
import re

def extract_flat_id(url):
    # Utilizar una expresión regular para extraer el ID del flat
    match = re.search(r"(MLC-\d+)", url)
    if match:
        return match.group(1)
    return np.nan


In [15]:
columns_to_check = [
    'General Expenses', 'Size of the flat', 'Bedrooms', 'Bathrooms', 'Seller',
    'metraje', 'sup_terraza', 'sup_util', 'ambientes', 'dormitorios', 'banos',
    'estacionamiento', 'bodegas', 'piso_unidad', 'cant_pisos', 'dept_piso',
    'antiguedad', 'tipo_depa', 'orientacion'
]

# Función para calcular la tasa de calidad
def calculate_quality_rate(row, columns):
    total_columns = len(columns)
    non_nan_count = row[columns].notna().sum()
    quality_rate = non_nan_count / total_columns
    return quality_rate


In [16]:
data['flat_id']=data['url'].map(extract_flat_id)
data['quality_rate'] = data.apply(calculate_quality_rate, columns=columns_to_check, axis=1)


In [17]:
data.columns

Index(['url', 'Name of the flat', 'Value', 'Currency', 'General Expenses',
       'Size of the flat', 'Bedrooms', 'Bathrooms', 'Seller', 'metraje',
       'sup_terraza', 'sup_util', 'ambientes', 'dormitorios', 'banos',
       'estacionamiento', 'bodegas', 'piso_unidad', 'cant_pisos', 'dept_piso',
       'antiguedad', 'tipo_depa', 'orientacion', 'Calle', 'Barrio', 'Comuna',
       'Ciudad', 'Dirección', 'Fecha_Publicacion', 'Description', 'new_orient',
       'map_depa', 'flat_id', 'quality_rate'],
      dtype='object')

In [18]:
data.head(4)

,url,Name of the flat,Value,Currency,General Expenses,Size of the flat,Bedrooms,Bathrooms,Seller,metraje,sup_terraza,sup_util,ambientes,dormitorios,banos,estacionamiento,bodegas,piso_unidad,cant_pisos,dept_piso,antiguedad,tipo_depa,orientacion,Calle,Barrio,Comuna,Ciudad,Dirección,Fecha_Publicacion,Description,new_orient,map_depa,flat_id,quality_rate
0,https://www.portalinmobiliario.com/MLC-2555490...,Dúplex Con Jardín I Espoz I Amplio Y Remodelado,24500,UF,440000.0,380.0,3.0,4.0,Vitacura Brokers,380.0,134.0,246.0,NaN,3.0,4.0,3.0,1.0,2.0,2.0,3.0,7.0,Departamento,NaN,Espoz 4000,Vitacura,Chile,Parque Bicentenario,"Espoz 4000, Vitacura, Chile, Parque Bicentenar...",2024-07-06,"Amplio e impecable Dúplex con jardín, en exclu...",NaN,Departamento,MLC-2555490854,0.894737
1,https://www.portalinmobiliario.com/MLC-2555757...,Tobalaba C_421813,4100,UF,45000.0,42.0,1.0,1.0,identidad verificada,42.0,NaN,37.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Metro Bilbao,Providencia,RM (Metropolitana),NaN,"Metro Bilbao, Providencia, RM (Metropolitana)",2024-07-06,Oportunidad Inversionistas!! esta arrenndado e...,NaN,NaN,MLC-2555757470,0.473684
2,https://www.portalinmobiliario.com/MLC-2555722...,Venta Depto Providencia 2d+2b+1est+1bod 80m2 H...,8000,UF,160000.0,80.0,2.0,2.0,identidad verificada,80.0,5.0,75.0,NaN,2.0,2.0,1.0,1.0,NaN,12.0,NaN,6.0,Departamento,S,Holanda 300 - 600,Providencia,Chile,Metro Tobalaba - Mall Costanera,"Holanda 300 - 600, Providencia, Chile, Metro T...",2024-07-06,Cuenta con:- 75m2 útiles y 5m2 de terraza con ...,S,Departamento,MLC-2555722768,0.842105
3,https://www.portalinmobiliario.com/MLC-2555742...,¡en Oportunidad! Amplio Dpto En Venta I Barrio...,9190,UF,220000.0,95.0,2.0,2.0,identidad verificada,95.0,14.0,81.0,NaN,2.0,2.0,1.0,1.0,10.0,11.0,6.0,5.0,Departamento,SO,Avenida Hernando De Aguirre 1191,Providencia,Chile,Las Lilas,"Avenida Hernando De Aguirre 1191, Providencia,...",2024-07-06,"Precioso departamento en venta, ubicado a solo...",SO,Departamento,MLC-2555742704,0.947368


In [19]:
def update_missing_pairs(df):
    """Update missing values in specified pairs of columns."""
    # Update 'Bedrooms' and 'dormitorios'
    df['Bedrooms'] = df['Bedrooms'].fillna(df['dormitorios'])
    df['dormitorios'] = df['dormitorios'].fillna(df['Bedrooms'])
    
    # Update 'Bathrooms' and 'banos'
    df['Bathrooms'] = df['Bathrooms'].fillna(df['banos'])
    df['banos'] = df['banos'].fillna(df['Bathrooms'])
    
    # Update 'Size of the flat' and 'metraje'
    df['Size of the flat'] = df['Size of the flat'].fillna(df['metraje'])
    df['metraje'] = df['metraje'].fillna(df['Size of the flat'])
    
    return df

In [20]:
def update_surface_areas(df):
    """Update metraje, sup_util, and sup_terraza columns based on given logic."""
    # Calculate 'metraje' if it is null
    df['metraje'] = df.apply(
        lambda row: row['sup_util'] + row['sup_terraza'] if pd.isnull(row['metraje']) and not pd.isnull(row['sup_util']) and not pd.isnull(row['sup_terraza']) else row['metraje'], 
        axis=1
    )

    # Calculate 'sup_util' if it is null
    df['sup_util'] = df.apply(
        lambda row: row['metraje'] - row['sup_terraza'] if pd.isnull(row['sup_util']) and not pd.isnull(row['metraje']) and not pd.isnull(row['sup_terraza']) else row['sup_util'], 
        axis=1
    )   

    # Calculate 'sup_terraza' if it is null
    df['sup_terraza'] = df.apply(
        lambda row: row['metraje'] - row['sup_util'] if pd.isnull(row['sup_terraza']) and not pd.isnull(row['metraje']) and not pd.isnull(row['sup_util']) else row['sup_terraza'], 
        axis=1
    )
    
    return df

In [21]:
data_clean=update_surface_areas(update_missing_pairs(data))

In [22]:
data_clean.head(4)

,url,Name of the flat,Value,Currency,General Expenses,Size of the flat,Bedrooms,Bathrooms,Seller,metraje,sup_terraza,sup_util,ambientes,dormitorios,banos,estacionamiento,bodegas,piso_unidad,cant_pisos,dept_piso,antiguedad,tipo_depa,orientacion,Calle,Barrio,Comuna,Ciudad,Dirección,Fecha_Publicacion,Description,new_orient,map_depa,flat_id,quality_rate
0,https://www.portalinmobiliario.com/MLC-2555490...,Dúplex Con Jardín I Espoz I Amplio Y Remodelado,24500,UF,440000.0,380.0,3.0,4.0,Vitacura Brokers,380.0,134.0,246.0,NaN,3.0,4.0,3.0,1.0,2.0,2.0,3.0,7.0,Departamento,NaN,Espoz 4000,Vitacura,Chile,Parque Bicentenario,"Espoz 4000, Vitacura, Chile, Parque Bicentenar...",2024-07-06,"Amplio e impecable Dúplex con jardín, en exclu...",NaN,Departamento,MLC-2555490854,0.894737
1,https://www.portalinmobiliario.com/MLC-2555757...,Tobalaba C_421813,4100,UF,45000.0,42.0,1.0,1.0,identidad verificada,42.0,5.0,37.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Metro Bilbao,Providencia,RM (Metropolitana),NaN,"Metro Bilbao, Providencia, RM (Metropolitana)",2024-07-06,Oportunidad Inversionistas!! esta arrenndado e...,NaN,NaN,MLC-2555757470,0.473684
2,https://www.portalinmobiliario.com/MLC-2555722...,Venta Depto Providencia 2d+2b+1est+1bod 80m2 H...,8000,UF,160000.0,80.0,2.0,2.0,identidad verificada,80.0,5.0,75.0,NaN,2.0,2.0,1.0,1.0,NaN,12.0,NaN,6.0,Departamento,S,Holanda 300 - 600,Providencia,Chile,Metro Tobalaba - Mall Costanera,"Holanda 300 - 600, Providencia, Chile, Metro T...",2024-07-06,Cuenta con:- 75m2 útiles y 5m2 de terraza con ...,S,Departamento,MLC-2555722768,0.842105
3,https://www.portalinmobiliario.com/MLC-2555742...,¡en Oportunidad! Amplio Dpto En Venta I Barrio...,9190,UF,220000.0,95.0,2.0,2.0,identidad verificada,95.0,14.0,81.0,NaN,2.0,2.0,1.0,1.0,10.0,11.0,6.0,5.0,Departamento,SO,Avenida Hernando De Aguirre 1191,Providencia,Chile,Las Lilas,"Avenida Hernando De Aguirre 1191, Providencia,...",2024-07-06,"Precioso departamento en venta, ubicado a solo...",SO,Departamento,MLC-2555742704,0.947368


In [23]:
data_clean['flat_id']=data_clean['url'].map(extract_flat_id)
data_clean['quality_rate'] = data_clean.apply(calculate_quality_rate, columns=columns_to_check, axis=1)


In [24]:
def update_antiguedad(row):
    """Update 'antiguedad' column based on given rules."""
    current_year = datetime.now(pytz.timezone('America/Santiago')).year
    
    antiguedad_value = row['antiguedad']
    
    # If antiguedad is NaN, empty, or None, do nothing
    if pd.isnull(antiguedad_value) or antiguedad_value == '':
        return antiguedad_value
    
    try:
        value = int(antiguedad_value)
    except ValueError:
        return np.nan

    # Replace values less than 0 or greater than 2030 with NaN
    if value < 0 or value > 2030:
        return np.nan
    
    # If value has more than 2 digits, calculate the age
    if len(str(value)) > 2:
        return current_year - value
    
    return value

In [25]:
# Apply the function to the 'antiguedad' column
data_clean['antiguedad'] = data_clean.apply(update_antiguedad, axis=1)

In [26]:
pd.set_option('display.max_rows',1000)

In [27]:
data_clean.sort_values(by='flat_id',ascending=False)

,url,Name of the flat,Value,Currency,General Expenses,Size of the flat,Bedrooms,Bathrooms,Seller,metraje,sup_terraza,sup_util,ambientes,dormitorios,banos,estacionamiento,bodegas,piso_unidad,cant_pisos,dept_piso,antiguedad,tipo_depa,orientacion,Calle,Barrio,Comuna,Ciudad,Dirección,Fecha_Publicacion,Description,new_orient,map_depa,flat_id,quality_rate
9145,https://www.portalinmobiliario.com/MLC-2597918...,Departamento Venta 4d L 2b Metro Pdevaldivia P...,4990,UF,110000.0,90.0,4.0,2.0,identidad verificada,90.0,0.0,90.0,NaN,4.0,2.0,NaN,NaN,10.0,24.0,NaN,55.0,Clásico,Nor-Poniente,Carlos Antunez - Metro Pedro De Valdivia L1 - ...,Pedro de Valdivia,Providencia,RM (Metropolitana),Carlos Antunez - Metro Pedro De Valdivia L1 - ...,2024-07-19,Torres de Carlos Antúnez un clásico de Provide...,NP,Clasico,MLC-2597918186,0.789474
9045,https://www.portalinmobiliario.com/MLC-2597918...,Departamento En Venta De 4 Dorm. En Providencia,4990,UF,110000.0,90.0,4.0,2.0,identidad verificada,90.0,0.0,90.0,NaN,4.0,2.0,NaN,NaN,10.0,24.0,NaN,55.0,Clásico,Nor-Poniente,Carlos Antunez - Metro Pedro De Valdivia L1 - ...,Pedro de Valdivia,Providencia,RM (Metropolitana),Carlos Antunez - Metro Pedro De Valdivia L1 - ...,2024-07-19,Torres de Carlos Antúnez un clásico de Provide...,NP,Clasico,MLC-2597918186,0.789474
9178,https://www.portalinmobiliario.com/MLC-2597909...,Fullremodelado Norte Vista Despejada Club De G...,12700,UF,180000.0,120.0,4.0,3.0,identidad verificada,120.0,9.0,111.0,4.0,4.0,3.0,1.0,1.0,12.0,13.0,2.0,2.0,-,Norte,Av Vitacura - Parque Bicentenario,Parque Bicentenario,Vitacura,RM (Metropolitana),"Av Vitacura - Parque Bicentenario, Parque Bice...",2024-07-19,VENTA - UF 12.700\rBicentenario - Club de Golf...,N,NaN,MLC-2597909674,1.000000
9064,https://www.portalinmobiliario.com/MLC-2597909...,Fullremodelado Norte Vista Despejada Club De G...,12700,UF,180000.0,120.0,4.0,3.0,identidad verificada,120.0,9.0,111.0,4.0,4.0,3.0,1.0,1.0,12.0,13.0,2.0,2.0,-,Norte,Av Vitacura - Parque Bicentenario,Parque Bicentenario,Vitacura,RM (Metropolitana),"Av Vitacura - Parque Bicentenario, Parque Bice...",2024-07-19,VENTA - UF 12.700\rBicentenario - Club de Golf...,N,NaN,MLC-2597909674,1.000000
9102,https://www.portalinmobiliario.com/MLC-2597909...,Departamento En Venta De 1 Dorm. En Providencia,3800,UF,58000.0,NaN,1.0,1.0,identidad verificada,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,5.0,NaN,NaN,NaN,-,NaN,Av Santa Maria,Bellavista,Providencia,RM (Metropolitana),"Av Santa Maria, Bellavista, Providencia, RM (M...",2024-07-19,Se vende hermoso y luminoso departamento a cua...,NaN,NaN,MLC-2597909118,0.421053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,https://www.portalinmobiliario.com/MLC-1500315...,Departamento En Venta De 3 Dorm. En Vitacura,12800,UF,180000.0,141.0,3.0,3.0,identidad verificada,141.0,11.0,130.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Departamento En Venta De 3 Dorm. En Vitacura,Parque Bicentenario,Vitacura,RM (Metropolitana),"Departamento En Venta De 3 Dorm. En Vitacura, ...",2024-07-07,Se vende espacioso departamento en la exclusiv...,NaN,NaN,MLC-1500315273,0.578947
387,https://www.portalinmobiliario.com/MLC-1500315...,Departamento En Venta De 3 Dorm. En Vitacura,12800,UF,180000.0,141.0,3.0,3.0,identidad verificada,141.0,11.0,130.0,3.0,3.0,3.0,2.0,1.0,NaN,3.0,2.0,40.0,Semi piso,NaN,Departamento En Venta De 3 Dorm. En Vitacura,Parque Bicentenario,Vitacura,RM (Metropolitana),"Departamento En Venta De 3 Dorm. En Vitacura, ...",2024-07-07,Se vende espacioso departamento en la exclusiv...,NaN,Semi Piso,MLC-1500315273,0.894737
707,https://www.portalinmobiliario.com/MLC-1500315...,Departamento En Venta De 3 Dorm. En Vitacura,12800,UF,180000.0,141.0,3.0,3.0,identidad verificada,141.0,11.0,130.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Departamento En Venta De 3 Dorm. En Vitacura,Parque Bicentenario,Vitacura,RM (Metropolitana),"Departamento En Venta De 3 Dorm. En Vita

In [28]:
data.columns

Index(['url', 'Name of the flat', 'Value', 'Currency', 'General Expenses',
       'Size of the flat', 'Bedrooms', 'Bathrooms', 'Seller', 'metraje',
       'sup_terraza', 'sup_util', 'ambientes', 'dormitorios', 'banos',
       'estacionamiento', 'bodegas', 'piso_unidad', 'cant_pisos', 'dept_piso',
       'antiguedad', 'tipo_depa', 'orientacion', 'Calle', 'Barrio', 'Comuna',
       'Ciudad', 'Dirección', 'Fecha_Publicacion', 'Description', 'new_orient',
       'map_depa', 'flat_id', 'quality_rate'],
      dtype='object')

In [31]:

def generate_column_dict(df):
    type_mapping = {
        'int64': 'INTEGER',
        'float64': 'REAL',
        'object': 'TEXT',
        'bool': 'BOOLEAN',
        'datetime64[ns]': 'TEXT'  # SQLite does not have a dedicated datetime type
    }
    
    column_dict = {col: type_mapping[str(dtype)] for col, dtype in df.dtypes.items()}
    return column_dict


# Generar el diccionario
process_col = generate_column_dict(data)
print(process_col)


{'url': 'TEXT', 'Name of the flat': 'TEXT', 'Value': 'INTEGER', 'Currency': 'TEXT', 'General Expenses': 'REAL', 'Size of the flat': 'REAL', 'Bedrooms': 'REAL', 'Bathrooms': 'REAL', 'Seller': 'TEXT', 'metraje': 'REAL', 'sup_terraza': 'REAL', 'sup_util': 'REAL', 'ambientes': 'REAL', 'dormitorios': 'REAL', 'banos': 'REAL', 'estacionamiento': 'REAL', 'bodegas': 'REAL', 'piso_unidad': 'REAL', 'cant_pisos': 'REAL', 'dept_piso': 'REAL', 'antiguedad': 'REAL', 'tipo_depa': 'TEXT', 'orientacion': 'TEXT', 'Calle': 'TEXT', 'Barrio': 'TEXT', 'Comuna': 'TEXT', 'Ciudad': 'TEXT', 'Dirección': 'TEXT', 'Fecha_Publicacion': 'TEXT', 'Description': 'TEXT', 'new_orient': 'TEXT', 'map_depa': 'TEXT', 'flat_id': 'TEXT', 'quality_rate': 'REAL'}


In [51]:

name_mapping={
'url':'url', 
'Name of the flat':'id_name', 
'Value':'valor', 'Currency':'moneda', 
'General Expenses':'gastos_comunes',
'Size of the flat':'sup_total',
'Bedrooms':'habitacion', 
'Bathrooms':'bano', 
'Seller':'vendedor', 
'metraje':'metraje',
'sup_terraza':'sup_terraza',
'sup_util':'sup_util', 
'ambientes':'ambientes', 
'dormitorios': 'dormitorios', 
'banos':'banos',
'estacionamiento':'estacionamiento', 
'bodegas':'bodega', 
'piso_unidad':'piso_unidad', 
'cant_pisos':'tot_pisos', 
'dept_piso':'dept_x_piso',
'antiguedad':'antiguedad', 
'tipo_depa':'tipo_depa', 
'orientacion':'orientacion', 
'Calle':'calle', 
'Barrio':'barrio', 
'Comuna':'comuna',
'Ciudad':'ciudad', 
'Dirección':'direccion', 
'Fecha_Publicacion':'fecha_scrap', 
'Description':'descripcion',
'flat_id':'id_publicacion', 
'quality_rate':'quality_scrap'
}

In [52]:

data_new=data.rename(columns=name_mapping)

In [54]:
data_new[[
    'url', 
    'id_name', 
    'valor',
    'moneda', 
    'gastos_comunes',
    'sup_total',
    'sup_terraza',
    'sup_util',  
    'dormitorios', 
    'banos',
    'estacionamiento', 
    'bodega', 
    'ambientes',
    'piso_unidad', 
    'tot_pisos', 
    'dept_x_piso',
    'antiguedad', 
    'tipo_depa', 
    'orientacion',
    'vendedor', 
    'calle', 
    'barrio', 
    'comuna',
    'ciudad', 
    'direccion', 
    'fecha_scrap', 
    'descripcion',
    'id_publicacion', 
    'quality_scrap']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9217 entries, 0 to 9216
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              9217 non-null   object 
 1   id_name          9217 non-null   object 
 2   valor            9217 non-null   int64  
 3   moneda           9217 non-null   object 
 4   gastos_comunes   8236 non-null   float64
 5   sup_total        9146 non-null   float64
 6   sup_terraza      9049 non-null   float64
 7   sup_util         9093 non-null   float64
 8   dormitorios      9191 non-null   float64
 9   banos            9198 non-null   float64
 10  estacionamiento  5961 non-null   float64
 11  bodega           4196 non-null   float64
 12  ambientes        4248 non-null   float64
 13  piso_unidad      3315 non-null   float64
 14  tot_pisos        3850 non-null   float64
 15  dept_x_piso      3138 non-null   float64
 16  antiguedad       4736 non-null   float64
 17  tipo_depa     

In [59]:
data[['url', 'Name of the flat', 'Value', 'Currency', 'General Expenses',
       'Size of the flat', 'Bedrooms', 'Bathrooms', 'Seller', 'metraje',
       'sup_terraza', 'sup_util', 'ambientes', 'dormitorios', 'banos',
       'estacionamiento', 'bodegas', 'piso_unidad', 'cant_pisos', 'dept_piso',
       'antiguedad', 'tipo_depa', 'orientacion', 'Calle', 'Barrio', 'Comuna',
       'Ciudad', 'Dirección', 'Fecha_Publicacion', 'Description']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9217 entries, 0 to 9216
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                9217 non-null   object 
 1   Name of the flat   9217 non-null   object 
 2   Value              9217 non-null   int64  
 3   Currency           9217 non-null   object 
 4   General Expenses   8236 non-null   float64
 5   Size of the flat   9146 non-null   float64
 6   Bedrooms           9191 non-null   float64
 7   Bathrooms          9198 non-null   float64
 8   Seller             9217 non-null   object 
 9   metraje            9156 non-null   float64
 10  sup_terraza        9049 non-null   float64
 11  sup_util           9093 non-null   float64
 12  ambientes          4248 non-null   float64
 13  dormitorios        9191 non-null   float64
 14  banos              9198 non-null   float64
 15  estacionamiento    5961 non-null   float64
 16  bodegas            4196 